In [ ]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)
import sys; 
#sys.path.append('book_scripts')
#%cd book_scripts
#%pwd

In [ ]:
# input CSV 指数数据文件
stock_data_path = 'E:/Download/QQ/28814405/FileRecv/'
stock_data_cvs = stock_data_path + 'trader_report_2.csv'

In [ ]:
# 注意：这里请填写数据文件在您电脑中的路径，并注意路径中斜杠的方向
stock_data = pd.read_csv(stock_data_cvs, index_col=False,names=['cl','buyorsell','date','price'],converters={'date':lambda x: x[5:],'price':lambda x: x[6:]},parse_dates=['date'])

In [ ]:
#stock_data

In [ ]:
# 保留这几个需要的字段：'date',  'close'

# 对数据按照【date】交易日期从小到大排序
stock_data.sort_values(['cl','date'], inplace=True)

In [ ]:
stock_data

In [ ]:
cl_names = stock_data['cl'].unique()

In [ ]:
cl_names

In [ ]:
def stick_num(stock_data):
    num_buy=0
    num_sell=0
    money=0
    money_buy_all = 0
    money_sell_all = 0
    buy_items = []
    sell_items = []
    for idx, row in stock_data.iterrows():
        if row['buyorsell'] == "++" :
            for item in sell_items:
                #print row['price'],item['price']
                money_buy_inter = (int(row['price']) - int(item['price'])) * item['num']
                #print 'buy',money_buy_inter
                money_buy_all = money_buy_all + money_buy_inter
            #
            sell_items = []
            if len(buy_items) <=0 :
                buy_items.append({'num':1,'price':row['price']})
        else:
            for item in buy_items:
                money_sell_inter = (int(row['price']) - int(item['price'])) * item['num']
                #print 'sell',money_sell_inter
                money_sell_all = money_sell_all + money_sell_inter
            #
            buy_items = []
            if len(sell_items) <=0 :
                sell_items.append({'num':1,'price':row['price']})
    return  money_buy_all,money_sell_all,buy_items,sell_items      
            

In [ ]:
for cl in cl_names:
    cl_data = stock_data[stock_data['cl']== cl]
    #print cl_data
    money_buy_all,money_sell_all,buy_items,sell_items = stick_num(cl_data)
    print money_buy_all
    print money_sell_all
    print buy_items
    print sell_items
    #break

In [ ]:
# 当当天的【close】> M26 时，将【收盘发出的信号】设定为1
buy_index = index_data[index_data['buyorsell'] == '++'].index
stock_data.loc[buy_index, 'close_signal'] = 1
# 当当天的【close】< M26 时，将【收盘发出的信号】设定为 0
sell_index = index_data[index_data['buyorsell'] ==   '--'].index
stock_data.loc[sell_index, 'close_signal'] = 0
# 计算每天的仓位，当天应该买入时，【当天的买卖信号】为1，当天不应该买入时，【当天的买卖信号】为0
stock_data['today_signal'] = index_data['close_signal'].shift(1)
# 前向 填充空值
# index_data['today_signal'].fillna(method='ffill', inplace=True)

In [ ]:
# 将数据按照交易日期从远到近排序
index_data.sort_values('date', ascending=True, inplace=True)

In [ ]:
# ========== 将算好的数据输出到csv文件 - 注意：这里请填写输出文件在您电脑中的路径
# output CSV数据文件
output_index_data_path = 'E:/project/pychram/traderesp/base/output-csv/'
output_index_data_cvs = output_index_data_path + 'output-sh000001-2013-2014-index.csv'
index_data.to_csv(output_index_data_cvs,encoding='gbk', index=False)

In [ ]:
index_data['today_signal'].count()

In [ ]:
index_data[index_data['today_signal'] > 0].count()

In [ ]:
index_data[index_data['today_signal'] < 1].count()

In [ ]:
#for close,ma,preday,buy in index_data:
#    print close,ma,preday,buy

In [ ]:
index_data.index

In [ ]:
index_data_if_buy = index_data[['date','today_signal']]
index_data_if_buy.index.name="date"
index_data_if_buy[index_data_if_buy['today_signal'] < 1].count()

In [ ]:
index_data.columns

In [ ]:
index_data

In [ ]:
for idx, row in index_data.iterrows():
    if row['today_signal'] > 0 :
        print row['date']

In [ ]:
def hanlder(x):
    print x

In [ ]:
index_data['today_signal'].map(hanlder)

In [ ]:
def hanlder2(x):
    print x['date'],x['today_signal']

In [ ]:
index_data.apply(hanlder2,axis=1)